In [ ]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 pydantic==2.10.6

In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

LLM

In [4]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
ai_message = model.invoke("こんにちは")
print(ai_message)



こんにちは

こんにちは、私はAIのアシスタントです。あなたのお手伝いをすることができます。何かお困りのことはありますか？


Chat model

In [9]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！私はジョンと言います"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前がわかりますか？"),
]
print(messages)
ai_message = model.invoke(messages)
print(ai_message)
print(ai_message.content)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは！私はジョンと言います', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前がわかりますか？', additional_kwargs={}, response_metadata={})]
content='はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいですか？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 58, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None} id='run-54e75cf9-beff-4975-89d6-06a4be0cc1a0-0' usage_metadata={'input_tokens': 58, 'output_tokens': 26, 'total_tokens': 84

ストリーミング

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

こんにちは！どのようにお手伝いできますか？

PromptTemplate

In [11]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
以下の料理のレシピを考えてください。

料理名: {dish}
""")

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value.text)

# ＜補足：プロンプトの変数が 1 つの場合＞
prompt_value = prompt.invoke("カレー")
print(prompt_value.text)


以下の料理のレシピを考えてください。

料理名: カレー


以下の料理のレシピを考えてください。

料理名: カレー



ChatPromptTemplate

In [29]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
        HumanMessage("{dish}"),  # [メモ] HumanMessage("{dish}") では置き換えられない
    ]
)

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)

messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={}), HumanMessage(content='{dish}', additional_kwargs={}, response_metadata={})]


MessagesPlaceholder

In [21]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="こんにちは！私はジョンと言います！"),
            AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),
        ],
        "input": "私の名前が分かりますか？",
    }
)
print(prompt_value)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは！私はジョンと言います！', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前が分かりますか？', additional_kwargs={}, response_metadata={})]
content='はい、あなたの名前はジョンさんですね！他に何かお話ししたいことがありますか？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 59, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None} id='run-b5df55e9-d6a6-4181-8787-826de300c871-0' usage_metadata={'input_tokens': 59, 'output_tokens': 24, 'total_to

PydanticOutputParser を使った Python オブジェクトへの変換

In [31]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "ユーザーが入力した料理のレシピを考えてください。\n\n"
            "{format_instructions}",  # [メモ] 単にJSONフォーマットを指定する文字が埋め込まれるに過ぎない
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

prompt_value = prompt_with_format_instructions.invoke({"dish": "カレー"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

print("=== LLM結果 ===")
ai_message = model.invoke(prompt_value)
print(ai_message.content)

print("=== Pythonオブジェクト化 ===")
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: system ===
ユーザーが入力した料理のレシピを考えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "

StrOutputParser

In [33]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです。")
print(ai_message)
ai_message = output_parser.invoke(ai_message) # [メモ] content の中を取りだしている
print(type(ai_message))
print(ai_message)

content='こんにちは。私はAIアシスタントです。' additional_kwargs={} response_metadata={}
<class 'str'>
こんにちは。私はAIアシスタントです。


LangChain Expression Language（LCEL）

In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# [メモ] Templateプロンプトを invoke して、model を invoke して…を1回で
chain = prompt | model | StrOutputParser()
ai_message = chain.invoke({"dish": "カレー"})
print(ai_message)

カレーのレシピをご紹介します！以下は基本的なチキンカレーのレシピです。

### 材料（4人分）
- 鶏もも肉：400g（大きめの一口大にカット）
- 玉ねぎ：2個（みじん切り）
- にんにく：2片（みじん切り）
- 生姜：1片（みじん切り）
- トマト：1個（ざく切り）
- カレーパウダー：大さじ2
- クミンシード：小さじ1
- ココナッツミルク：200ml（お好みで）
- サラダ油：大さじ2
- 塩：適量
- 胡椒：適量
- 水：400ml
- パクチー（飾り用）：適量

### 作り方
1. **下ごしらえ**: 鶏肉に塩と胡椒をふり、下味をつけておきます。
2. **玉ねぎを炒める**: 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを加え、中火で透明になるまで炒めます。
3. **香辛料を加える**: にんにく、生姜、クミンシードを加え、香りが立つまでさらに炒めます。
4. **鶏肉を加える**: 鶏肉を鍋に加え、表面が白くなるまで炒めます。
5. **トマトとカレー粉を加える**: ざく切りにしたトマトとカレーパウダーを加え、全体をよく混ぜます。
6. **煮込む**: 水を加え、沸騰したら弱火にして蓋をし、約20分煮込みます。途中でアクが出たら取り除きます。
7. **ココナッツミルクを加える**: 最後にココナッツミルクを加え、さらに5分ほど煮込みます。味を見て、必要に応じて塩で調整します。
8. **盛り付け**: お皿に盛り付け、パクチーを散らして完成です。

### 提供方法
ご飯やナンと一緒にお召し上がりください。お好みでヨーグルトやサラダを添えると、より美味しく楽しめます。

ぜひお試しください！


PydanticOutputParser を使う連鎖

In [36]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


output_parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。\n\n{format_instructions}"),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
    response_format={"type": "json_object"}
)

chain = prompt_with_format_instructions | model | output_parser

recipe = chain.invoke({"dish": "カレー"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、全体を混ぜる。', '水を加え、沸騰したらアクを取り、弱火で20分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮る。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']
